In [1]:
%load_ext sql
%sql sqlite:///complaint.db
import time
%sql drop index if exists helpful_index;
%sql analyze

Done.
Done.


/Library/Python/2.7/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/Library/Python/2.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


[]

Let's play with the [consumer complaint database](https://catalog.data.gov/dataset/consumer-complaint-database) from data.gov

In [2]:
%sql select count(*) from complaints;

Done.


count(*)
818801


In [4]:
%sql select * from complaints limit 5;

Done.


Complaint_ID,Product,Subproduct,Issue,Subissue,State,ZIP_code,Submitted_via,Date_received,Date_sent_to_company,Company,Company_response,Timely_response,Consumer_disputed
None,None,None,None,None,None,None,None,None,None,None,None,None,None
1431865,Consumer loan,Vehicle loan,Managing the loan or lease,,NJ,08736,Web,06/22/2015,06/22/2015,Nissan Motor Acceptance Corporation,In progress,Yes,
1431374,Debt collection,Medical,Disclosure verification of debt,Not given enough info to verify debt,WI,54140,Web,06/22/2015,06/22/2015,"RSI Enterprises, Inc.",Closed with explanation,Yes,
1431251,Mortgage,Conventional fixed mortgage,"Loan modification,collection,foreclosure",,MO,63368,Web,06/22/2015,06/22/2015,Flagstar Bank,In progress,Yes,
1431743,Debt collection,Medical,Cont'd attempts collect debt not owed,Debt is not mine,WA,98055,Web,06/22/2015,06/22/2015,"P.S.C., Inc",Closed with explanation,Yes,Yes


The syntax to create a view is below
> create index (index_name) on (table)(attributes)

In [9]:
%%sql drop index if exists zip_index;
create index zip_index on complaints(zip_code);

Done.
Done.


[]

In [5]:
%%sql
select zip_code, count(*) as c 
from complaints group by zip_code 
order by c desc limit 5;

Done.


ZIP_code,c
,9234
48382,1842
33071,1242
33173,774
76116,742


In [10]:
%%sql select product, count(*)
from complaints c1 where ZIP_CODE = '48382'
group by product;

Done.


Product,count(*)
Bank account or service,3
Credit card,3
Credit reporting,4
Debt collection,2
Mortgage,908
Student loan,1


Let's get down to business... Indexing complainers!

For pedagogical purposes, we'll use _likelihood_ function:
  * likelihood(X,Y) does not change X's value 
  * instead it tells the DBMS the fraction of calls on which we believe X holds.
      * 0.0 is lowest, 1.0 is highest.
  * It's a form of _hint_ for the optimizer

In [10]:
%%sql 
explain query plan 
select product, count(*) from complaints c1 
where likelihood(ZIP_CODE = '48382', 1.0) group by product;

Done.


selectid,order,from,detail
0,0,0,SCAN TABLE complaints AS c1
0,0,0,USE TEMP B-TREE FOR GROUP BY


In [11]:
%%sql 
explain query plan 
select product, count(*) from complaints c1 
where likelihood(ZIP_CODE = '48382', 0.0) group by product;

Done.


selectid,order,from,detail
0,0,0,SEARCH TABLE complaints AS c1 USING INDEX zip_index (ZIP_code=?)
0,0,0,USE TEMP B-TREE FOR GROUP BY


So... does it make a difference in execution time? 

... Well, some it's a small database in memory...

In [19]:
start_scan = time.time()
%sql select product, count(*) from complaints c1 where likelihood(ZIP_CODE = '48382', 1.0) group by product;
scan_time = time.time() - start_scan

start_index = time.time()
%sql select product, count(*) from complaints c1 where likelihood(ZIP_CODE = '48382', 0.0) group by product;
index_time  = time.time() - start_index
print("time for scan={0} time for index={1}".format(scan_time, index_time))
print("\t So about {0} times faster".format(scan_time/index_time))

Done.
Done.
time for scan=0.00245881080627 time for index=0.00234699249268
	 So about 1.04764323446 times faster


<center>Covering Indexes</center>

We say that an index is _covering_ if all the needed columns are in the index.
 * Needed columns is the union of those in WHERE _and_ SELECT.
 * Means that the query can be answere using the index--without reading the table!

In [13]:
%%sql drop index if exists helpful_index;
create index helpful_index on 
complaints(zip_code, product, complaint_id);

Done.
Done.


[]

Which plan happens for this query?
> select product, count(*)<br>
> from complaints <br>
> where likelihood(ZIP_CODE = '48382', 1.0)
> <br>group by product;

NB: Using zip_index, this would be a scan!

In [15]:
%%sql
explain query plan select product, count(*) 
from complaints c1 
where likelihood(ZIP_CODE = '48382', 1.0) group by product;

Done.


selectid,order,from,detail
0,0,0,SEARCH TABLE complaints AS c1 USING COVERING INDEX helpful_index (ZIP_code=?)


* The index is _smaller_ in absolute data size.
* The DBMS knows this, and it picks the right index!

Summary
-------

* This notebook walks through index creation 
* That index selection is done in a cost based way.
* Introduced the notion of covering index.
* Demonstrated that not only keys to answer query, but attributes returned important component of selection. 